In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid

from network import FeedForwardDenseNetwork
import activation
import metrics
import losses
import normalizers

# Generate data

In [2]:
def generate_data(size: int):
    data = np.random.uniform(low=0, high=1, size=(2, size))
    data = data * np.random.choice([-1, 1], size=(2, size))
    labels = (np.sign(data[0]) == np.sign(data[1])).astype(int)
    return data, labels

In [3]:
train_data, train_labels = generate_data(50_000)
train_data.shape, train_labels.shape

((2, 50000), (50000,))

In [4]:
test_data, test_labels = generate_data(10_000)
test_data.shape, test_labels.shape

((2, 10000), (10000,))

# Instantiate and fit the models

In [5]:
NEURONS = [2, 4, 1]
METRIC = metrics.Accuracy
LOSS = losses.MSE

params_dict = {
    'activation': [activation.Relu, activation.Sigmoid],
    'normalizer': [None, normalizers.L1, normalizers.L2],
    'learning_rate': [0.001, 0.005, 0.01]
}
params_dict

{'activation': [activation.Relu, activation.Sigmoid],
 'normalizer': [None, normalizers.L1, normalizers.L2],
 'learning_rate': [0.001, 0.005, 0.01]}

In [6]:
param_grid = pd.DataFrame(ParameterGrid(params_dict))
param_grid

,activation,learning_rate,normalizer
0,<class 'activation.Relu'>,0.001,None
1,<class 'activation.Relu'>,0.001,<class 'normalizers.L1'>
2,<class 'activation.Relu'>,0.001,<class 'normalizers.L2'>
3,<class 'activation.Relu'>,0.005,None
4,<class 'activation.Relu'>,0.005,<class 'normalizers.L1'>
5,<class 'activation.Relu'>,0.005,<class 'normalizers.L2'>
6,<class 'activation.Relu'>,0.010,None
7,<class 'activation.Relu'>,0.010,<class 'normalizers.L1'>
8,<class 'activation.Relu'>,0.010,<class 'normalizers.L2'>
9,<class 'activation.Sigmoid'>,0.001,None


In [7]:
model_dict = {
    'model': [],
    'loss': [],
    'accuracy': []
}

for i in param_grid.index:
    # instantiate the model
    model = FeedForwardDenseNetwork(
        neurons=NEURONS,
        activation=param_grid.loc[i, 'activation'],
        metric=METRIC,
        loss=LOSS,
        normalizer=param_grid.loc[i, 'normalizer']
    )

    # fit the model
    model.fit(
        X=train_data, 
        Y=train_labels, 
        learning_rate=param_grid.loc[i, 'learning_rate'],
        verbose=False
    )

    # evaluate the model
    loss, accuracy = model.evaluate(test_data, test_labels)
    model_dict['model'].append(model)
    model_dict['loss'].append(loss)
    model_dict['accuracy'].append(accuracy)

    print(f"{i}: {loss = }, {accuracy = }") 

0: loss = 0.4182, accuracy = 0.5818
1: loss = 0.4268, accuracy = 0.5732
2: loss = 0.507, accuracy = 0.493
3: loss = 0.4059, accuracy = 0.5941
4: loss = 0.154, accuracy = 0.846
5: loss = 0.5003, accuracy = 0.4997
6: loss = 0.3254, accuracy = 0.6746
7: loss = 0.082, accuracy = 0.918
8: loss = 0.5087, accuracy = 0.4913
9: loss = 0.493, accuracy = 0.507
10: loss = 0.3859, accuracy = 0.6141
11: loss = 0.493, accuracy = 0.507
12: loss = 0.6025, accuracy = 0.3975
13: loss = 0.4718, accuracy = 0.5282
14: loss = 0.507, accuracy = 0.493
15: loss = 0.4884, accuracy = 0.5116
16: loss = 0.462, accuracy = 0.538
17: loss = 0.5827, accuracy = 0.4173


In [8]:
model_grid = pd.DataFrame(model_dict)
model_grid

,model,loss,accuracy
0,<network.FeedForwardDenseNetwork object at 0x0...,0.4182,0.5818
1,<network.FeedForwardDenseNetwork object at 0x0...,0.4268,0.5732
2,<network.FeedForwardDenseNetwork object at 0x0...,0.5070,0.4930
3,<network.FeedForwardDenseNetwork object at 0x0...,0.4059,0.5941
4,<network.FeedForwardDenseNetwork object at 0x0...,0.1540,0.8460
5,<network.FeedForwardDenseNetwork object at 0x0...,0.5003,0.4997
6,<network.FeedForwardDenseNetwork object at 0x0...,0.3254,0.6746
7,<network.FeedForwardDenseNetwork object at 0x0...,0.0820,0.9180
8,<network.FeedForwardDenseNetwork object at 0x0...,0.5087,0.4913
9,<network.FeedForwardDenseNetwork object at 0x0...,0.4930,0.5070


In [9]:
best_model = model_grid.loc[model_grid['accuracy'].idxmax()]
best_model.model.summary()
best_model.loss, best_model.accuracy

Number of layers: 3
Number of neurons per layer:
	1: 2
	2: 4
	3: 1
Activation: relu
Metric: accuracy
Loss: mean_sqared_error
Normalization: L1


(0.082, 0.918)